In [43]:
import pandas as pd
import numpy as np
import ftfy
import unicodedata
import re
from lxml import etree
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
train_labels = pd.read_csv('data/train_labels.csv')
train_labels.head()

,article_id,dataset_id,type
0,10.1002_2017jc013030,https://doi.org/10.17882/49388,Primary
1,10.1002_anie.201916483,Missing,Missing
2,10.1002_anie.202005531,Missing,Missing
3,10.1002_anie.202007717,Missing,Missing
4,10.1002_chem.201902131,Missing,Missing


In [81]:
train_labels[train_labels['dataset_id'] ==  'GO0034774']

,article_id,dataset_id,type


In [ ]:
list(train_labels[train_labels['article_id'] == '10.1371_journal.pcbi.1011828']['dataset_id'])

['E-PROT-100',
 'E-PROT-101',
 'E-PROT-102',
 'E-PROT-103',
 'E-PROT-104',
 'E-PROT-105',
 'E-PROT-106',
 'E-PROT-107',
 'E-PROT-108',
 'E-PROT-109',
 'E-PROT-110',
 'E-PROT-23',
 'PXD001676',
 'PXD002137',
 'PXD005693',
 'PXD005709',
 'PXD010458',
 'PXD014511',
 'PXD019504',
 'PXD020454',
 'PXD031556',
 'PXD032899']

In [56]:
train_labels[train_labels['article_id'] == '10.1021_acsomega.3c06074'].shape

(11, 3)

In [5]:
train_labels.shape

(1028, 3)

In [6]:
train_labels['dataset_id'].isna().sum()

np.int64(0)

In [7]:
train_labels[train_labels['dataset_id'] == 'Missing'].shape

(309, 3)

In [8]:
train_labels['dataset_id'].nunique()

720

In [9]:
train_labels.loc[train_labels['dataset_id'].str.contains('https://doi.org/10'), 'dataset_id'] = 1
train_labels.loc[train_labels['dataset_id'] == 'Missing', 'dataset_id'] = 0
train_labels['dataset_id'].nunique()

396

In [10]:
print(list(train_labels['dataset_id'].unique()))


[1, 0, '5VA1', 'CHEMBL1097', 'CHEMBL1257578', 'CHEMBL1257821', 'CHEMBL1782574', 'CHEMBL195180', 'CHEMBL2424928', 'CHEMBL3422978', 'CHEMBL390649', 'CHEMBL572163', 'CHEMBL70', 'IPR000884', 'IPR001124', 'IPR001577', 'IPR002477', 'IPR002889', 'IPR004007', 'IPR004302', 'IPR004785', 'IPR004911', 'IPR004963', 'IPR004981', 'IPR005524', 'IPR006201', 'IPR008139', 'IPR009613', 'IPR011040', 'IPR011124', 'IPR012338', 'IPR016201', 'IPR017920', 'IPR021345', 'IPR028730', 'IPR029723', 'IPR029787', 'IPR033396', 'IPR036452', 'PF00348', 'PF00484', 'PF01493', 'PF05834', 'PF15461', 'GSE130386', 'SRP184152', 'EMPIAR-10005', 'EMPIAR-10017', 'EMPIAR-10028', 'EMPIAR-10081', 'EMPIAR-10093', 'EMPIAR-10345', 'EMPIAR-10387', 'EMPIAR-10406', 'EMPIAR-10532', 'EMPIAR-10590', 'EMPIAR-10852', 'EMPIAR-11057', 'ENSBTAG00000011038', 'ENSBTAG00000013718', 'ENSBTAG00000017121', 'ENSBTAG00000017131', 'ENSBTAG00000021275', 'ENSBTAG00000047833', 'ENSOARG00000003950', 'ENSOARG00000012128', 'ENSOARG00000012835', 'ENSOARG000000137

In [11]:
len(list(train_labels['dataset_id'].unique()))

396

In [12]:
li = list(train_labels['dataset_id'].unique())
li = [item for item in li if item not in [0, 1]]



In [13]:
samn = [item for item in li if item.startswith('SAMN')]
print(len(samn))
print(samn)

41
['SAMN07159041', 'SAMN07159044', 'SAMN07159045', 'SAMN08172567', 'SAMN08199278', 'SAMN09534374', 'SAMN12620691', 'SAMN12620692', 'SAMN12620693', 'SAMN12620694', 'SAMN12620697', 'SAMN12620700', 'SAMN12620701', 'SAMN12620702', 'SAMN12620703', 'SAMN12620704', 'SAMN17525956', 'SAMN16233641', 'SAMN16233642', 'SAMN16233643', 'SAMN16233644', 'SAMN16233645', 'SAMN16233646', 'SAMN16233647', 'SAMN16233648', 'SAMN16233649', 'SAMN16233650', 'SAMN16233651', 'SAMN16233652', 'SAMN16233653', 'SAMN16233654', 'SAMN16233655', 'SAMN16233656', 'SAMN16233657', 'SAMN16233658', 'SAMN16233659', 'SAMN16233660', 'SAMN16233661', 'SAMN16233662', 'SAMN16233663', 'SAMN16233664']


In [14]:
samns = re.findall(r'\bSAMN\d{8}\b', ' '.join(li))
print(len(samns))
print(samns)

41
['SAMN07159041', 'SAMN07159044', 'SAMN07159045', 'SAMN08172567', 'SAMN08199278', 'SAMN09534374', 'SAMN12620691', 'SAMN12620692', 'SAMN12620693', 'SAMN12620694', 'SAMN12620697', 'SAMN12620700', 'SAMN12620701', 'SAMN12620702', 'SAMN12620703', 'SAMN12620704', 'SAMN17525956', 'SAMN16233641', 'SAMN16233642', 'SAMN16233643', 'SAMN16233644', 'SAMN16233645', 'SAMN16233646', 'SAMN16233647', 'SAMN16233648', 'SAMN16233649', 'SAMN16233650', 'SAMN16233651', 'SAMN16233652', 'SAMN16233653', 'SAMN16233654', 'SAMN16233655', 'SAMN16233656', 'SAMN16233657', 'SAMN16233658', 'SAMN16233659', 'SAMN16233660', 'SAMN16233661', 'SAMN16233662', 'SAMN16233663', 'SAMN16233664']


In [15]:
che = [item for item in li if item.startswith('CHEMBL')]
print(len( che))
print(che)

29
['CHEMBL1097', 'CHEMBL1257578', 'CHEMBL1257821', 'CHEMBL1782574', 'CHEMBL195180', 'CHEMBL2424928', 'CHEMBL3422978', 'CHEMBL390649', 'CHEMBL572163', 'CHEMBL70', 'CHEMBL1568820', 'CHEMBL1652119', 'CHEMBL198796', 'CHEMBL243644', 'CHEMBL2436978', 'CHEMBL3109593', 'CHEMBL3311228', 'CHEMBL3752642', 'CHEMBL3960662', 'CHEMBL402487', 'CHEMBL403', 'CHEMBL4065616', 'CHEMBL408500', 'CHEMBL561', 'CHEMBL64391', 'CHEMBL76', 'CHEMBL3347550', 'CHEMBL3347551', 'CHEMBL3733336']


In [ ]:


def clean_text(text):
    # 1. Fix encoding issues like mojibake or smart quotes
    text = ftfy.fix_text(text)

    # 2. Normalize ligatures and full-width characters
    text = unicodedata.normalize("NFKC", text)

    # 3. Remove non-printable control characters (ASCII 0–31 and 127)
    text = re.sub(r'[\x00-\x1F\x7F]', '', text)

    # 4. Remove unnecessary backslashes before quotes (like d\'Océanographie → d'Océanographie)
    text = re.sub(r'\\([\'"“”‘’`´])', r'\1', text)

    # 5. Add space between glued lowercase-uppercase boundaries (e.g., datasetEuropean → dataset European)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

    # 6. Insert space before URLs (e.g., doihttps://... → doi https://...)
    text = re.sub(r'(?<=[a-zA-Z])(?=https?://)', ' ', text)

    # 7. Replace all weird dash characters with standard hyphen
    text = re.sub(r'[\u2010\u2011\u2012\u2013\u2014\u2212]', '-', text)

    # 8. Add space after punctuation if not followed by space
    text = re.sub(r'([,;!?)\]\}])(?=\S)', r'\1 ', text)

    # 9.Remove all symbols **except** alphanumeric, hyphen, underscore, dot, and space
    text = re.sub(r'[^\w\s\.\-_\/]', ' ', text)

    # 10.Collapse repeated valid symbols, while preserving `//`
    text = re.sub(r'-{2,}', '-', text)
    text = re.sub(r'_{2,}', '_', text)
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'/{3,}', '//', text)  # Keep `//` but collapse longer

    # 11. Flatten all whitespace (spaces, tabs, newlines) into a single space
    text = re.sub(r'\s+', ' ', text)

    # 12. Remove any leftover leading/trailing space
    return text.strip()

In [17]:


def extract_text_from_xml(xml_path):
    parser = etree.XMLParser(recover=True)
    tree = etree.parse(xml_path, parser)
    root = tree.getroot()
    text_parts = []
    for elem in root.iter():
        if elem.text and elem.text.strip():
            text_parts.append(elem.text.strip())
        if elem.tail and elem.tail.strip():
            text_parts.append(elem.tail.strip())
    
    text =  ''.join(text_parts)
    return clean_text(text)

In [33]:
textx = extract_text_from_xml('data/train/XML/10.7717_peerj.10452.xml')
textx[:10000]

'open_access-//NLM//DTD JATS Z39.96 Journal Publishing DTD v1.1 20151215//ENpublic JATS-journalpublishing1.dtd1.1jp2nlmx2.xsl1Peer JPeer Jpeerjpeerj Peer J2167-8359Peer J Inc.San Diego USA78799491045210.7717/peerj.10452Agricultural Science Biotechnology Microbiology Molecular Biology Mycology Bacterial community diversity lignocellulose components and histological changes in composting using agricultural straws for Agaricus bisporusproduction Song Tingting1Shen Yingyue1Jin Qunli1Feng Weilin1Fan Lijun1Cao Guangtian15a1903025 cjlu.edu.cn2Cai Weimingcaiwm0527 126.com11Institute of Horticulture Zhejiang Academy of Agricultural Sciences Hanzghou Zhejiang China2China Jiliang University College of Standardisation Hangzhou Zhejiang China Arora Gunjan92202120219e1045210820209112020 2021 Song et al.2021Song et al.This is an open access article distributed under the terms of the Creative Commons Attribution License which permits unrestricted use distribution reproduction and adaptation in any med

In [19]:
import fitz
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return clean_text(text)

In [34]:
textp = extract_text_from_pdf('data/train/PDF/10.7717_peerj.10452.pdf')
textp[:10000]

'Submitted 10 August 2020Accepted 9 November 2020Published 9 February 2021Corresponding authors Guangtian Cao 15a1903025 cjlu.edu.cn Weiming Cai caiwm0527 126.com Academic editor Gunjan Arora Additional Information and Declarations can be found onpage 14DOI 10.7717/peerj.10452Copyright2021 Song et al.Distributed under Creative Commons CC-BY 4.0OPEN ACCESSBacterial community diversity lignocellulose components and histologicalchanges in composting using agriculturalstraws for Agaricus bisporus production Tingting Song1 Yingyue Shen1 Qunli Jin1 Weilin Feng1 Lijun Fan1 Guangtian Cao2 and Weiming Cai11 Institute of Horticulture Zhejiang Academy of Agricultural Sciences Hanzghou Zhejiang China2 China Jiliang University College of Standardisation Hangzhou Zhejiang China ABSTRACTAgricultural straws AS may serve as potential base-substances in the productionof Agaricus bisporus. Six AS that occur across China were investigated in a two-stagecomposting experiment lignocellulose components AS mo

In [ ]:

def extract_context(text, pattern, window=30):
    """
    Extract context around regex matches from given text.

    Parameters:
    - text (str): Input document text.
    - pattern (str): Regex pattern to match.
    - window (int): Number of words before and after match to include in context.

    Returns:
    - List of dicts with 'match' and 'context' keys.
    """

    matches = []

    # Tokenize the full text once
    tokens = text.split()

    # Map character index → token index
    char_to_token = {}
    index = 0
    for i, tok in enumerate(tokens):
        index = text.find(tok, index)
        for j in range(index, index + len(tok)):
            char_to_token[j] = i
        index += len(tok)

    # Find matches using re.finditer()
    for match in re.finditer(pattern, text):
        raw_match = match.group()
        cleaned_match = re.sub(r'[\s\-]', '', raw_match)  # Remove spaces/dashes inside ID

        start_char = match.start()

        if start_char in char_to_token:
            token_index = char_to_token[start_char]
            start = max(0, token_index - window)
            end = min(len(tokens), token_index + window + 1)
            context_window = tokens[start:end]

            matches.append({
                'match': cleaned_match,
                'context': ' '.join(context_window)
            })

    return matches

In [ ]:
extract_context(textp, r'SAMN\d{8}')


In [ ]:
extract_context(textp, r'EPI_ISL_\d+')

In [ ]:
extract_context(textp, r'SAMN\d{8}')

[{'match': 'SAMN16233641',
  'context': 'draft.DNA Deposition The following information was supplied regarding the deposition of DNA sequences The 16s RNA sequences of compost for Agaricus bisporus production are available at Gen Bank PRJNA664798.Bio Sample SAMN16233641 SAMN16233642 SAMN16233643 SAMN16233644 SAMN16233645 SAMN16233646 SAMN16233647 SAMN16233648 SAMN16233649 SAMN16233650 SAMN16233651 SAMN16233652 SAMN16233653 SAMN16233654 SAMN16233655 SAMN16233656 SAMN16233657 SAMN16233658 SAMN16233659 SAMN16233660 SAMN16233661 SAMN16233662 SAMN16233663 SAMN16233664.The 16s RNA sequences of compost for Agaricus'},
 {'match': 'SAMN16233642',
  'context': 'Deposition The following information was supplied regarding the deposition of DNA sequences The 16s RNA sequences of compost for Agaricus bisporus production are available at Gen Bank PRJNA664798.Bio Sample SAMN16233641 SAMN16233642 SAMN16233643 SAMN16233644 SAMN16233645 SAMN16233646 SAMN16233647 SAMN16233648 SAMN16233649 SAMN16233650 SAM

In [41]:
len(extract_context(textp, r'SAMN\d{8}'))

24

In [42]:
samns = re.findall(r'SAMN[\s\-]?\d{8}', textx)
print(len(samns))
print(samns)

24
['SAMN16233641', 'SAMN16233642', 'SAMN16233643', 'SAMN16233644', 'SAMN16233645', 'SAMN16233646', 'SAMN16233647', 'SAMN16233648', 'SAMN16233649', 'SAMN16233650', 'SAMN16233651', 'SAMN16233652', 'SAMN16233653', 'SAMN16233654', 'SAMN16233655', 'SAMN16233656', 'SAMN16233657', 'SAMN16233658', 'SAMN16233659', 'SAMN16233660', 'SAMN16233661', 'SAMN16233662', 'SAMN16233663', 'SAMN16233664']


In [37]:
samns = re.findall(r'SAMN[\s\-]?\d{8}', textp)
print(len(samns))
print(samns)

24
['SAMN16233641', 'SAMN16233642', 'SAMN16233643', 'SAMN16233644', 'SAMN16233645', 'SAMN16233646', 'SAMN16233647', 'SAMN16233648', 'SAMN16233649', 'SAMN16233650', 'SAMN16233651', 'SAMN16233652', 'SAMN16233653', 'SAMN16233654', 'SAMN16233655', 'SAMN16233656', 'SAMN16233657', 'SAMN16233658', 'SAMN16233659', 'SAMN16233660', 'SAMN16233661', 'SAMN16233662', 'SAMN16233663', 'SAMN16233664']


In [25]:
import sys
print(sys.executable)

/Users/aswinbenny/Downloads/projects/make_data_count/dataset-reference-finder/.venv/bin/python
